In [ ]:
import requests
import plotly.graph_objects as go
from ipywidgets import widgets
import pandas as pd
from traitlets import HasTraits, Dict, observe, default
from IPython.core.display import display

In [ ]:
import sys

if sys.platform == 'win32':
    import asyncio
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

# Indicateurs de service d'eau et d'assainissement par commune en France 

* Cet outil présente l'évolution des indicateurs de service (D), des indicateurs de performance de performance (P) et des Variables de performance (VP) des services d'alimentation en eau potable (AEP), d'assainissement collectif (AC) et d'assinissement non-collectif (ANC) pour les communes françaises
* Une description détaillée des indicateurs est consultable sur le site de l'**[Observatoire des services d'eau et d'assainissement](http://www.services.eaufrance.fr/indicateurs/indicateurs)**. 
* Quand il existe plusieurs services publics sur une même commune, un minimum, maximum et moyenne sont calculés 

In [2]:
# Source url and api gets
source = 'https://hubeau.eaufrance.fr/api'
get_communes = '/v0/indicateurs_services/communes'
get_indicateurs = '/v0/indicateurs_services/indicateurs'
get_services = '/v0/indicateurs_services/services'

communes_csv_url = "https://raw.githubusercontent.com/julienenilorac/FR_water_kpis/master/communes-01012019.csv"
indicateurs_csv_url = "https://raw.githubusercontent.com/julienenilorac/FR_water_kpis/master/Liste%20des%20indicateurs%20de%20performance%20140318.csv"

In [3]:
# Request function that retrieves data from API in the form of a dictionary
# The expected data is a list of dict pey year
# If the data key is missing from the resulting json, returns an empty list

def get_data_from_api(code_commune,type_service,size=20):
    str_req = '?code_commune={}&format=json&size={}&type_service={}'.format(code_commune,size,type_service)
    url = source + get_communes + str_req
    response = requests.get(url)
    data = response.json()['data']
    # data = data.get('data',[])      
    return data

In [4]:
# Function that builds a {year : {indicators : value}} dict from raw data,
# re-arrange data in year ascending order,
# drops empty columns (indicators),
# and returns a DataFrame

def get_graph_data(data):
    graph_data = dict()
    
    for year_data in data:    
        annee = year_data['annee']
        indicator = year_data['indicateurs']
        graph_data[annee] = indicator
        
    df = pd.DataFrame.from_dict(graph_data,orient='index')
    df = df.sort_index(axis=0 ,ascending=True)
    df = df.dropna(axis=1,how='all')
    return df

In [5]:
# Initial request parameters
initial_code = 97415
size = 20
initial_service = 'AEP'

In [6]:
# Initial request
data = get_data_from_api(code_commune=initial_code,
                         type_service=initial_service,
                         size=20)

df = get_graph_data(data)

In [7]:
# Creates a DataFrame listing all cities along with their INSEE Code
df_cities= pd.read_csv(communes_csv_url)
# Adds a colums to df to be used in Combobox to search for a city / INSEE code
df_cities['combo'] = df_cities['ncc'] + " (" + df_cities['com'] + ")"

In [8]:
# Creates a DataFrame listing all indicators with their description and unit
df_indicators = pd.read_csv(indicateurs_csv_url)
df_indicators['combo'] = df_indicators['Code'] + " - " + df_indicators['Description_simple'] 
df_indicators = df_indicators.set_index("Code")
indicator_descriptions = df_indicators.to_dict(orient="dict")["combo"]
df = df.rename(columns=indicator_descriptions)
df_indicators = df_indicators.set_index("combo")

In [9]:
# Dictionnary to store the resulting DataFrame
df_dict={
    'new': df,
    'old': df
}

In [10]:
# Creates a traitlets class to store dict data
class cls_Storage(HasTraits):
    data = Dict()

In [11]:
app_storage = cls_Storage(data=df_dict)

In [12]:
# function that adds a description of each indicator (if available) to the data df
def add_indicator_description(dataframe):
    pass

In [13]:
# Graph layout for unavailable data

unavailable_data_template = go.layout.Template()
unavailable_data_template.layout.annotations = [
    dict(
        name="data_unavailable",
        text="Données non-disponibles pour cette commune",
        textangle=-30,
        opacity=0.3,
        font=dict(color="black", size=20),
        xref="paper",
        yref="paper",
        x=0.5,
        y=0.5,
        showarrow=False,
    )
]

In [14]:
# Instantiates combobox and dropdown widgets

opts_lst = list(df_cities['combo'])

combobox = widgets.Combobox(
    options = opts_lst,
    ensure_option = True,
    value = 'SAINT PAUL (97415)',
    placeholder='Nom de commune ou code INSEE',
    description='Commune:'
)

dropdown_service = widgets.Dropdown(
    options=['AEP','AC','ANC'],
    value='AEP',
    description='Service:',
    disabled=False
)

dropdown_indicators = widgets.Dropdown(
    options=list(df_dict['old'].columns),
    value=list(df_dict['old'].columns)[0],
    description='Indicateur:',
    disabled = False
)

HTML_indicator = widgets.HTML(
    value=df_indicators.loc[dropdown_indicators.value,"Description"],
    description='Indicateur: ',
)

container_1 = widgets.HBox([combobox,dropdown_service])
container_2 = widgets.HBox([dropdown_indicators])
container_3 = widgets.HBox([HTML_indicator])

# Assign an empty figure widget with a single trace
trace1 = go.Scatter(x=df_dict['old'].index,y=df_dict['old'][dropdown_indicators.value], opacity=0.75, name='Indicateur')
g = go.FigureWidget(data=[trace1],
                    layout=go.Layout(
                        title=dict(
                            text=dropdown_indicators.value
                        ),
                        barmode='overlay',
                        template = None
                    ))

In [15]:
# Debug views for callbacks
debug_view_1 = widgets.Output(layout={'border': '1px solid black'})
debug_view_2 = widgets.Output(layout={'border': '4px solid black'})

In [16]:
# Update request
# @debug_view_1.capture(clear_output=True)
def get_data(change):
    
    # if changes comes from dropdown service
    if change.new == dropdown_service.value:
        INSEE_code = df_cities[df_cities['combo'].isin([combobox.value])].iloc[0]['com']
        service = change.new
    
    # if changes comes from dropdown service
    if change.new == combobox.value:
        INSEE_code = df_cities[df_cities['combo'].isin([change.new])].iloc[0]['com']
        service = dropdown_service.value      
    
    data = get_data_from_api(INSEE_code,service)     

    df_temp = get_graph_data(data=data)
    df_temp = df_temp.rename(columns=indicator_descriptions)
    df_dict['new'] = df_temp
    app_storage.data['new'] = df_temp
    print(app_storage.data['new'])
    

In [17]:
# Update the indicator dropdown option list when a new service is selected
# @debug_view_2.capture(clear_output=True)
def update_dropdown(change):
    if app_storage.data['new'].empty:
        dropdown_indicators.options = ['-']
        HTML_indicator.value = "-"
    else:
        print(list(app_storage.data['new'].columns))
        dropdown_indicators.options = list(app_storage.data['new'].columns) 
        dropdown_indicators.value = dropdown_indicators.options[0]
        HTML_indicator.value = df_indicators.loc[dropdown_indicators.value,"Description"]
    
      

In [18]:
# Update graph when user selects a new indicator
# @debug_view_2.capture(clear_output=True)
def update_graph(change):    
  
    with g.batch_update():
        
            if app_storage.data['new'].empty:
                
                g.data[0].y = pd.DataFrame()
                g.update_layout(template=unavailable_data_template)
        
            else:
                g.layout.template='plotly'
                g.data[0].y = app_storage.data['new'][change.new]      
                g.layout.barmode = 'overlay'
                g.layout.title = {"text": change.new}
                g.layout.xaxis.title = 'Année'
                g.layout.yaxis.title = df_indicators.loc[change.new,"Unité"]             
            
                


In [19]:
# Callbacks from update_requests
dropdown_service.observe(get_data, names="value")
combobox.observe(get_data, names="value")

# Callbacks from app_storage
# app_storage.observe(update_dropdown,names="data")

# Callbacks from update_dropdown
dropdown_service.observe(update_dropdown,names="value")
combobox.observe(update_dropdown,names="value")

# Callbacks from update graph
dropdown_indicators.observe(update_graph, names="value")


In [20]:
# Display all widgets
w = widgets.VBox([container_1,
              container_2,
              g,
              container_3,
              # debug_view_1,
              # debug_view_2
             ])

display(w)

Source: <https://hubeau.eaufrance.fr/page/api-indicateurs-services>

Contact: <julien.enilorac@jenipy.com>